In [1]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [35]:
news_df = pd.read_csv('train.csv')

In [37]:
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [41]:
news_df.shape

(20800, 5)

In [43]:
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [45]:
news_df = news_df.fillna(' ')

In [47]:
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [49]:
news_df['content'] = news_df['author']+' '+news_df['title']

In [51]:
news_df

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel Abrams Macy...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex Ansary NATO, Russia To Hold Parallel Exer..."


In [55]:
news_df['content'][20796]

'Benjamin Hoffman N.F.L. Playoffs: Schedule, Matchups and Odds - The New York Times'

In [61]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]'," ",content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [65]:
# Install NLTK if not installed (only run if needed)
!pip install nltk


In [67]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pandas as pd


In [69]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PRISCA.S\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [71]:
ps = PorterStemmer()

def stemming(content):
    content = content.lower()  # Convert to lowercase
    content = content.split()  # Tokenize (split into words)
    content = [ps.stem(word) for word in content if word not in stopwords.words('english')]  
    return ' '.join(content)  # Join words back into a sentence


In [73]:
news_df['content'] = news_df['content'].astype(str).apply(stemming)


In [74]:
print(news_df['content'].head())


0    darrel lucu hous dem aide: didn’t even see com...
1    daniel j. flynn flynn: hillari clinton, big wo...
2              consortiumnews.com truth might get fire
3    jessica purkiss 15 civilian kill singl us airs...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object


In [75]:
news_df['content']

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object

In [79]:
X = news_df['content'].values
y = news_df['label'].values

In [81]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)


In [83]:
print(X)

  (0, 809)	0.3646500188253278
  (0, 3403)	0.33756896138985654
  (0, 3952)	0.2266469969205269
  (0, 4763)	0.33044571153796654
  (0, 4995)	0.2512923264945339
  (0, 5256)	0.27468869329117757
  (0, 6433)	0.21422587910261737
  (0, 8832)	0.20534182453318398
  (0, 9692)	0.22757176689298134
  (0, 10747)	0.26822209263186303
  (0, 11072)	0.33384522056560495
  (0, 16473)	0.23676064517956455
  (0, 19097)	0.27315635150958634
  (1, 2258)	0.2928176012009572
  (1, 2713)	0.15460118725006144
  (1, 3100)	0.3870784468942128
  (1, 3778)	0.19062686807106288
  (1, 4728)	0.26268668599849243
  (1, 7101)	0.711483310803025
  (1, 8608)	0.19815023888659125
  (1, 20416)	0.29951908908156866
  (2, 3933)	0.31669937181571
  (2, 4162)	0.4602325600950218
  (2, 6968)	0.39293503470255664
  (2, 7650)	0.34605253138342823
  :	:
  (20797, 8874)	0.21736692401926294
  (20797, 10431)	0.22258731789449554
  (20797, 11179)	0.36057430237452964
  (20797, 11767)	0.2945753459972408
  (20797, 11844)	0.17405416749037161
  (20797, 12685)	0

In [85]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=2)

In [87]:
X_train.shape

(16640, 21822)

In [89]:
model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

In [91]:
# on training set
train_y_pred = model.predict(X_train)
print(accuracy_score(train_y_pred,Y_train))

0.9865985576923076


In [93]:
# on testing set
testing_y_pred = model.predict(X_test)
print(accuracy_score(testing_y_pred,Y_test))

0.9774038461538461


In [95]:
input_data = X_test[10]
prediction = model.predict(input_data)

In [97]:
if prediction[0] == 0:
    print('The News Is Real')
else:
    print('The News is Fake')

The News Is Real


In [99]:
news_df['content'][2]

'consortiumnews.com truth might get fire'